In [41]:
a = {1, 2, 3}

In [42]:
a.remove(2)

In [75]:
class A:
    li = [1, 2, 3]

    def set_li(self, li):
        self.li = li

    def summon_actions(self):
        return self.li 
a = A()

In [76]:
class Buff:
    def apply_buff(self, target):
        f = target.summon_actions
        def _func():
            li = f()
            return [x*x for x in li]
        setattr(target, "summon_actions", _func)

In [82]:
A.li = [1, 2, 4]

In [83]:
a = A()

In [84]:
a.li

[1, 2, 4]

In [80]:
a.summon_actions()

[1, 4, 1, 4, 16, 4, 9]

In [375]:
class MyDecorator:
    def __init__(self, function, *args):
        print(*args)
        self.function = function
        self.count = 0
        self.buff = []
     
    def __call__(self, *args, **kwargs):
        self.count+=1
        # We can add some code 
        # before function call
        print("call ke pehle!!!!")
        self.function(self, *args, **kwargs)
        print("call ke baad!!!")
        print("Count is {}, buffs are {}".format(self.count, self.buff))

    def add_buff(self, buff):
        self.buff.append(buff)
 
        # We can also add some code
        # after function call.
 
class A:
    def __init__(self, *args) -> None:
        print(*args)
 
    @MyDecorator
    def method(self):
        print("hehe")

# adding class decorator to the function
@MyDecorator
def function():
    print("GeeksforGeeks")


@MyDecorator
def func():
    print("GeekskiMKB!!!")

In [376]:
a = A()

In [377]:
a.method()

call ke pehle!!!!
hehe
call ke baad!!!
Count is 1, buffs are []


In [43]:
function.add_buff(1)
function.add_buff(2)
function.add_buff(4)
func.add_buff(1)

In [443]:
from typing import Any


class BuffManager:

    def __init__(self, function, instance = None, *args, **kwargs):
        self.function = function
        if instance is not None:
            self.instance = instance
            self.buffs = list()
        else:
            self.instances = {}
        self.is_property = False
        if isinstance(self.function, property):
            self.is_property = True
        

    def get_or_create_instance(self, _obj, *args, **kwargs):
        if _obj not in self.instances:
            self.instances[_obj] = BuffManager(self.function, _obj)
        return self.instances[_obj]


    def __get__(self, instance, owner):
        if instance is None:
            return self
        print("Call hota bhai!!!")
        return self.get_or_create_instance(instance)
        

    def add_buff( self, buff, _obj = None, *args, **kwargs):
        if _obj == None:
            _obj = self
        _obj.buffs.append(buff)
        
    def remove_buff(self, buff, _obj = None, *args, **kwargs):
        if _obj == None:
            _obj = self
        _obj.buffs.remove(buff)

    def __call__(self, *args, **kwargs):
        
        f = self.function
        for item in self.buffs:
            f = item(f)
        return f(self.instance,*args, **kwargs)
        
def managed_by_buff(function, *args, **kwargs):
    if isinstance(function, property):
        print("property hai!!!!")
        buff_manager = PropertyBuffManager(function)
    else:
        buff_manager = BuffManager(function)
    return buff_manager

class PropertyBuffManager(BuffManager):
    def __get__(self, instance, owner):
        print("This is called!!!!", owner)
        print(instance)
        if instance is None:
            return self
        return self.get_or_create_instance(instance).function.fget(instance)

def f1(func):

    def _func(_obj, x, *args, **kwargs):
        print("Pre f1, adding 1")
        _output = func(_obj, x + 1, *args, **kwargs)
        print("Post f1, printing output", _output)
        return _output
    return _func
    

def f2(func):
    def _func(_obj, x, *args, **kwargs):
        print("Pre f2, adding 4")
        _output = func(_obj, x + 4, *args, **kwargs)
        print("Post f2, printing output", _output)
        return _output
    return _func

def f3(func):
    def _func(_obj, x, *args, **kwargs):
        print("Pre f3, doing nothing")
        _output = func(_obj, x, *args, **kwargs)
        print("Post f3, multiplying output by 5")
        _output *= 5
        return _output
    return _func

def property_f1(func):

    def _func(_obj, x, *args, **kwargs):
        print("Pre f1, adding 1")
        _output = func(_obj, x + 1, *args, **kwargs)
        print("Post f1, printing output", _output)
        return _output
    return _func
    

def f2(func):
    def _func(_obj, x, *args, **kwargs):
        print("Pre f2, adding 4")
        _output = func(_obj, x + 4, *args, **kwargs)
        print("Post f2, printing output", _output)
        return _output
    return _func

def f3(func):
    def _func(_obj, x, *args, **kwargs):
        print("Pre f3, doing nothing")
        _output = func(_obj, x, *args, **kwargs)
        print("Post f3, multiplying output by 5")
        _output *= 5
        return _output
    return _func


class T:
    x = 2

    @managed_by_buff
    @property
    def y(self):
        return self.x + 1
    
    @managed_by_buff
    def fn(self, x):
        return x    

class Q(T):
    pass

property hai!!!!


In [444]:
t = Q()
q = Q()
t.fn

Call hota bhai!!!


In [447]:
t.y

This is called!!!! <class '__main__.Q'>


3

In [446]:
T.y.add_buff(f1, T.y.get_or_create_instance(t))

This is called!!!! <class '__main__.T'>
None
This is called!!!! <class '__main__.T'>
None


In [450]:
T.y.get_or_create_instance(t).buffs

This is called!!!! <class '__main__.T'>
None


[<function __main__.f1(func)>]

In [452]:
T.y.get_or_create_instance(q).buffs

This is called!!!! <class '__main__.T'>
None


[]

In [429]:
t.fn.add_buff(f2)

Call hota bhai!!!


In [335]:
t.fn.remove_buff(f1)

Called Get
Returning instance object for  <__main__.Q object at 0x7f809af310>


ValueError: list.remove(x): x not in list

In [428]:
q.fn.buffs

Call hota bhai!!!


[<function __main__.f2(func)>]

In [430]:
t.fn.buffs

Call hota bhai!!!


[<function __main__.f2(func)>]

In [41]:
q.fn.buffs

{<function __main__.f1(func)>}

In [242]:

dir(t.fn.function)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [267]:
getattr(t, t.fn.function.__name__)

In [467]:

q.fn.function

<function __main__.T.fn(self, x)>

In [ ]:
t.


